# 使い方

YAIBA-VRCで取得したログデータを分析し、ヒストグラム・ヒートマップ・統計データ・動画を生成し、ZIPファイルで出力します。

## 手順

1. プライバシーポリシーと同意事項を読む
2. ライブラリのインストール
3. ファイルアップロード
4. 処理開始

## プライバシーポリシーと同意事項

【重要】 プライバシーとデータの取り扱いについて
- 本ツールは、VRChatワールドギミック「YAIBA-VRC」によって位置情報を回転情報の取得に**同意したユーザーのログ**のみを扱うことを想定しています。
- ファイルをアップロードすることにより、データがGoogle Colaboratoryのサーバーに送信され、処理されることに同意したものとみなします。
  これは**第三者提供**に準ずる行為となりますので、取り扱うデータの内容には十分ご注意ください。

In [ ]:
# @title ライブラリのインストール

# フォントファイルのインストール
!apt-get -qq -y update
!apt-get -qq -y install fonts-noto-cjk fonts-ipafont-gothic

# YAIBA
!pip install --quiet git+https://github.com/ScienceAssembly/YAIBA.git

# YAIBA-BI
!pip install --quiet git+https://github.com/earl-klutz/YAIBA-democratization.git@develop_gerusuraimu  # Release作成後は「@latest」や「@stable」に変更

In [ ]:
# @title ファイルアップロード

# ライブラリimport
import os
import shutil
from typing import Dict
from google.colab import files
import ipywidgets as widgets
from IPython.display import display
from yaiba_bi import core
from yaiba_bi import colab_ui


def get_display_box(e: Exception) -> widgets.HTML:
    contents = widgets.HTML(
        value=f"<div style='padding:10px; border:2px solid orange; background:#FFF3E0; color:#E65100;'>"
              f"<b>エラー:</b> {str(e)}</div>"
    )
    return contents


# ディレクトリ初期設定
colab_ui.initialize()


# ファイルアップロード処理
uploaded: Dict[str, bytes] = files.upload(colab_ui.INPUT)
if len(uploaded) != 1:
    # 処理に失敗した場合はデータ全消去（情報保護）
    colab_ui.finalize()
    exception = ValueError("ログファイルは一つだけ指定してください")
    display(get_display_box(exception))
    raise exception

# ファイルパス取得
path = list(uploaded.keys())[0]
print(f"path: {path}")

# LogDataオブジェクト作成
try:
    log_data: core.LogData = core.load(path)
except ValueError as e:
    # 処理に失敗した場合はデータ全消去（情報保護）
    colab_ui.finalize()
    display(get_display_box(e))
    raise e
else:
    # inputディレクトリ削除
    # 生ログデータは個人情報が含まれていることがあるので、必ず削除する。
    shutil.rmtree(colab_ui.INPUT)

In [ ]:
# @title 処理開始

# 動画作成処理を実行するかを決めるBool値
# @markdown 動画を作成する場合はチェックをつけてください
is_generate_movie = False  # @param {"type": "boolean"}

if os.path.isfile("./output.zip"):
    os.remove("./output.zip")

# 処理
try:
    # 処理内容を記述
    position = log_data.get_position()
    attendance = log_data.get_attendance()
    area = log_data.get_area()

    heatmap_generator = core.HeatmapGenerator(area)
    heatmap_generator.run(position.copy(), "heatmap", colab_ui.OUTPUT)

    io_params = core.HistIOParams(out_dir=colab_ui.OUTPUT)
    core.run_histogram_mvp(df=position.copy(), io=io_params, output_basename="histogram")

    if is_generate_movie:
        movie_io_params = core.MovieIOParams(out_dir="./YAIBA_data/output")
        core.run_movie_xz(log_data.get_position(), io=movie_io_params)

    event_date = log_data.get_position()["event_day"].unique()[0]
    render_config = core.RenderConfig(str(event_date), "sample")
    trajectory_config = core.TrajectoryConfig()

    visualizer = core.EventLogVisualizer(render_config, trajectory_config)
    visualizer.run(log_data.get_attendance(), log_data.get_position(), log_data.get_area())
except Exception as e:
    display(get_display_box(e))
    raise e

# ZIPファイル作成
shutil.make_archive("output", format="zip", root_dir=colab_ui.OUTPUT)

# 作成したZIPファイルをダウンロードする
files.download("output.zip")

# ディレクトリの後処理
colab_ui.finalize()
colab_ui.initialize()